###### Set up and environments 

1. The code below is split between a jupyter notebook and .py script (Aider.py) ,which would be imported below for data preprocessing ,scaling,train test split ,running the model algoritms and generating the output with required metrics by returning a model object. 

2. The jupyter notebook and the pythn script were run both on Google colab and Nimblebox.ai cloud platforms to check the speed and reliability of the platform or the task at hand.

###### Import the required libraries

In [ ]:
import Aider
import pandas as pd
import numpy as np
#import objgraph -- memory allocation checks 
#import joblib -- importing the model objects to .pkl files

###### Read the Train data and merge it 

In [ ]:
frequency_domain=pd.read_csv('frequency_domain_features_train.csv')
heart_rate_non_linear=pd.read_csv('heart_rate_non_linear_features_train.csv')
time_domain=pd.read_csv('time_domain_features_train.csv')

###### Data Merging and Preprocessing

1. We get the merged data from the preprocessing file 

2. Aider .py script was written to practise clean coding of keeping different tasks in different sheets 

3. Aider does the following actions under preprocessing 

4. Default checks for the data - shape,summary stats,null value checks ,datatype info
5. Preprocessing - Merge dataframes and apply one hot encoding on the data ,drop unnecessary columns

In [ ]:
data=Aider.preprocessing(frequency_domain,heart_rate_non_linear,time_domain)

###### Very basic EDA 

In [ ]:
## Plotting Correlations

In [ ]:
## Dependent variable distribution 

In [ ]:
from scipy import stats

###### Outcomes/Insights 

1. There's a few highly correlated variables in teh dataset and in hindsight dropping none of which helped improved the model,we have retained all teh features in teh model as teh MAE seemed to be dropping otherwise

2. The distribution of the target variable is checked and we see that is very skewed and may need transformation and hence box-cox transformation was tried upon teh dataset.

###### Box Cox Transformation of the variables

###### Outcome of Box cox transformation

1. All the variables in teh data seemed to be highly skewed ,a trial was ocnducted with a box cox transformation of the variables that had skewness>0.75 .
2. The transformation of teh target variable and the other variables showed no significant uplift in the improvement of the model scores .
3. The changes were therefore made redundant keeping in mind the 'No-impact' of the skewness changes.

###### Deleting the dataframes to free up memory

In [ ]:
del frequency_domain
del heart_rate_non_linear
del time_domain

In [ ]:
# Regression Libraries 

#Importing required libraries
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor,BaggingRegressor
from sklearn import model_selection
from sklearn.svm import SVR
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline


In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

###### Modeling Steps followed

1. The Train test split and scaling code is in the Aider python script under the function 'split_train_test' 

2. There's a modeling helper function written to assist the process of running an algorithm clutter free . 

3. The function tracks the time elapsed during the run of each algorithm ,fitting the train data and predict it on the test data . The metrics printed along with train and test MAE are r2 and a model object is returned as an output of the helper function

###### Multiple modeling experiments on different regressors - Checking basic models`

Bagging Regressor

In [ ]:
bgre=BaggingRegressor(random_state=42)
x_train,x_test,y_train,y_test,sc=Aider.split_train_test(data,test_ratio=0.2)
bg_model=Aider.run_algorithm(bgre,x_train,y_train,x_test,y_test,'BaggingRegressor')

LightGBM Regressor 

Gradient Boost Regressor 

In [ ]:
Random Forest Regressor 

In [ ]:
rfre=RandomForestRegressor(random_state=42)
x_train,x_test,y_train,y_test,sc=Aider.split_train_test(data,test_ratio=0.2)
rfre_model=Aider.run_algorithm(rfre,x_train,y_train,x_test,y_test,'RFRegressor')

In [ ]:
#Reduced learning rate to 0.1 - improved performance of MAE

In [ ]:
#Reduced learning rate to 0.01 - degraded performance of MAE

In [ ]:
#Reduced learning rate to 0.05 - degraded performance of MAE

Decision Trees

Support Vector Regressor

Neural Networks 

###### Features Importnaces

###### PCA 

###### Outcomes - Checking basic Models 

1. There were a few Linear models with polynomial degrees tried ,none of which matched the low  MAE score of Random Forest ,Bagging Regressor AdaBoost Regressor

2. Multiple base models for bagging and boosting were tried to see the outputs with low MAE's .

3. Random Forest ,Baging Regressor ,Adaboost Regressor were clear winners 

In [ ]:
### Experiment - 1

In [ ]:
etre=ExtraTreesRegressor(random_state=42,n_jobs=-1,n_estimators=100)
x_train,x_test,y_train,y_test,sc=Aider.split_train_test(data,test_ratio=0.2)
etre_model=Aider.run_algorithm(etre,x_train,y_train,x_test,y_test,'ETRegressor')

`Extra Trees Regressor seems like the most efficient model ,with the least MAE score so far,we will proceed with our experiments in tuning it and ensembling it as well as stacking it`

In [ ]:
### Experiment - 2 

In [ ]:
etre_2=ExtraTreesRegressor(random_state=42,n_jobs=-1,n_estimators=1000)
x_train,x_test,y_train,y_test,sc=Aider.split_train_test(data,test_ratio=0)
etre_model=Aider.run_algorithm(etre_2,x_train,y_train,x_test,y_test,'ETRegressor')

In [ ]:
### Training the model on 100% data after a test on 80 Train and 20 Test 

In [ ]:
etre_4=ExtraTreesRegressor(random_state=42,n_jobs=-1,n_estimators=1330,max_depth=50)
x_train,x_test,y_train,y_test,sc=Aider.split_train_test(data,test_ratio=0)
etre_model=Aider.run_algorithm(etre_4,x_train,y_train,x_test,y_test,'ETRegressor')

In [ ]:
### Experiment - 3 

## Checking the combination of various parameters and their effects on the model output i.e Train and Test MAE values.

In [ ]:
### Experiment - 4 
## Can job lib pkl files store the model ? 

In [ ]:
#joblib.dump(etre_model,'etre_model_1330.pkl')

###### Ensemble -  Adaboost & Extra Trees Regressor

###### Lessons learnt -  reflection 

1. The ensemble produces a good output however fails the Kernel/Kernel
dies when number of estimators increase ,either in the bse estimator 
or Adaboost regressor .
2. Model objects are too huge to  be stored in .pkl files 
3. Lack of computational efficiency rules this model out even though it produces very good results .

###### Model Stacking -  Voting Regressor + Stacking Regressor

Stacking Regressor

###### Outcome

1. The output was not satisfactory enough to consider Voting Regressor as a stacking option.

2. The stacking regressor for RF and Bagging Regressor did not generate as good an output as expected.

###### Ensemble- Bagging+ExtraTrees Regressor


###### Outcome

The output was not satisfactory and hence we decide to not use this ensemble 

In [ ]:
## Deleting 'data' as we do not need it anymore and clearing some memory

In [ ]:
del data

##### Test Data

`Steps followed`

1. Merge the 3 test datasets and create a single dataframe
2. Preprocess the dataframe to keep it model ready.
3. Predict using the model that worked best from the above trials
4. Send the results to a dataframe and convert the dataframe to a .csv file 

In [ ]:
# Read and create a test file ,by merging the .csv's

In [ ]:
frequency_domain_test=pd.read_csv('frequency_domain_features_test.csv')
heart_rate_non_linear_test=pd.read_csv('heart_rate_non_linear_features_test.csv')
time_domain_test=pd.read_csv('time_domain_features_test.csv')

In [ ]:
temp=frequency_domain_test.merge(heart_rate_non_linear_test,on='uuid')
datatest1 = temp.merge(time_domain_test,on='uuid')

In [ ]:
# Applying preprocessing from the 'Aider' file we imported 

In [ ]:
datatest=Aider.preprocessing(frequency_domain_test,heart_rate_non_linear_test,time_domain_test)

In [ ]:
# Deleting the dataframes to clear some memory so the kernel does not die.

In [ ]:
del frequency_domain_test
del heart_rate_non_linear_test
del time_domain_test

In [ ]:
data1=datatest

In [ ]:
# Scaling ,predicting on the test data.

In [ ]:
x_val=sc.transform(data1)

In [ ]:
del datatest

In [ ]:
y_pred_val=etre_4.predict(x_val)

In [ ]:
# Loadig the results to a new dataframe

In [ ]:
submission_df=pd.DataFrame()

In [ ]:
datatest.columns

In [ ]:
submission_df['uuid']=datatest1['uuid']
submission_df['HR']=y_pred_val

In [ ]:
submission_df.head()

In [ ]:
# Convert the dataframe to a .csv file and saving to the current folder

In [ ]:
submission_df.to_csv('submission_ETRE-1330.csv',index=False)

In [ ]:
submission_df.shape

###### Submissions

1.Multiple submissions were made with the outputs for Randomforest regressor ,ensemble models a)Adaboost Regressor + Random Forest b)AdaBoost Regressor + ExtraTrees Regressor c)Bagging Regressor (tuned)  

2.Extratrees Regressor was tried with multiple tuning parameters and a tuned model of 1330 estimators produced the least MAE ,compared to all other stacked and ensemble models .There can be an ensemble of Adaboost and ExtraTrees which could work very well ,but the computational efficiency with the Kernel dying and memory errors throwing,the least score we could get to was 0.0014xx

In [ ]:
############################## End ############################